In [31]:
from datetime import date, datetime, timedelta

#### liste des fonctions à créer
- Échange
- Ballotage/Libérer joueur
- Commutations
    - Blessures
- Repêchages

In [3]:
#créer la table DRAFT en SQL avec les choix au repêchage
conn = sqlite3.connect('C:/Users/huber/OneDrive/NHL/DbMatchs.db')
#choix = pd.read_csv("C:\dev\pool-de-hockey\src\data\listeChoix.csv", sep=";")
#choix.to_sql('DRAFT', conn, if_exists='replace', index = False)
def execSQL(sql, conn, write=False):
    c = conn.cursor()
    print(c.execute(sql))
    if write:
        conn.commit()
    else:
        print(c.fetchall())
    c.close()

In [3]:
#DO NOT EXECUTE créer une fausse table ALIGNEMENTS POUR FAIRE LES TESTS
#alignements = pd.read_csv("alignements.csv", sep=";")
#alignements.to_sql('copALIGN', conn, if_exists='replace', index = False)

In [1]:
#Donc pour les choix:
    # - faire la fonction qui update la colonne 'Appartient à' dans la table DRAFT et l'ajouter dans la fonction échange
    # - ajouter les choix dans la fonction ajouterRegistre


#Ensuite:
    # - fonction getId(prénom, nom):
            #fonction qui retourne l'id d'un joueur dans la table JOUEURS
    # - fonction undoÉchange(idTrade):
            #fonction qui annule un échange
    # - optimiser la fonction générale:
            #-faire qu'on rentre le prénom/nom d'un joueur et les choix dans *1* csv
            #-le csv est uploadé en code
            #-le code sépare joueurs et choix et envoient les ids (grâce à getId) dans les csvs TradeChoix et                       TradeJoueurs (qui seraient probablement pus des csv rendu là)
            #-le reste de la fonction s'effectue
    # - vérificateur de lineup:
        #-serait intéressant d'être capable de le jumeler à la fonction échange et que dans le cas où un échange                rend un lineup non-conforme, un paramètre indique si on continue l'échange quand même (exemple l'équipe             en défaut compte mettre un joueur au ballotage immédiatement après) où on le break (et donc l'annule                avec undoÉchange)

In [3]:
#importer les éléments joueurs de l'échange
tradeJoueurs = pd.read_csv("TradeJoueurs.csv", sep=";")
ids = tradeJoueurs["idNHL"]

#importer les éléments choix de l'échange
tradeChoix = pd.read_csv("TradeChoix.csv", sep=";")
idChoix = tradeChoix["idChoix"]


#requête qui va chercher la ligne actuelle d'un joueur dans ALIGNEMENTS
def joueurLive(id):
    requete = "SELECT * FROM copALIGN WHERE idNHL == " + id + " AND dateFin == '2100-01-01'"
    df = pd.read_sql(requete, conn)
    return df

#fonction qui termine la période actuelle des joueurs de l'échange
def finirPeriode(joueurs, dat):
    dat = datetime.strptime(dat, "%Y-%m-%d")
    dat = dat - timedelta(days=1)
    dat = dat.strftime("%Y-%m-%d")
    dat = str(dat)
    for i in joueurs:
        str_i = str(i)
        df = joueurLive(str_i)
        idAlign = df.iloc[0]["idAlignement"]
        idAlign = str(idAlign)
        req = "UPDATE copALIGN SET dateFin = '" + dat + "' WHERE idAlignement == " + idAlign
        execSQL(req, conn, write=True)


#écrire la nouvelle ligne du joueur
def nouvellePeriode(id, dat):
    strId = str(id)
        # idAlignement : max de idAlignement + 1
    reqIdAlign = "SELECT MAX(idAlignement) FROM copALIGN"
    W_idAlign = pd.read_sql(reqIdAlign, conn)
    W_idAlign = W_idAlign.iloc[0]["MAX(idAlignement)"] + 1
        # idPooler : aller le chercher dans TradeJoueurs.csv avec idNHL
    is_idNHL = tradeJoueurs["idNHL"] == id
    joueurTrade = tradeJoueurs[is_idNHL]
    W_idPooler = joueurTrade.iloc[0]["equipeRecoit"]
        # prenomJoueur : aller le chercher dans JOUEURS avec idNHL
    reqPrenom = "SELECT prenomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_prenom = pd.read_sql(reqPrenom, conn)
    W_prenom = W_prenom.iloc[0]["prenomJoueur"]
        # nomJoueur : aller le chercher dans JOUEURS avec idNHL
    reqNom = "SELECT nomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_nom = pd.read_sql(reqNom, conn)
    W_nom = W_nom.iloc[0]["nomJoueur"]
        # idNHL : paramètre de la fonction
        # statutJoueur : aller le chercher dans joueurLive(id)
    df = joueurLive(strId)
    W_statut = df.iloc[0]["statutJoueur"]
        # dateDebut : paramètre de la fonction
        # dateFin : '2100-01-01'
    W_dateFin = "2100-01-01"

    d = {"idAlignement" : [W_idAlign], "idPooler" : [W_idPooler], "prenomJoueur" : [W_prenom], "nomJoueur" : [W_nom], "idNHL" : [id], "statutJoueur" : [W_statut], "dateDebut" : [dat], "dateFin" : [W_dateFin], "pjActuels" : [0], "butsActuels" : [0], "assistActuels" : [0], "pointsActuels": [0]}
    periode = pd.DataFrame(data=d)
    periode.to_sql("copALIGN", conn, if_exists="append", index=False)
    ok = W_prenom + " " + W_nom
    return ok

#fonction get noms de joueur
def getNom(id):
    strId = str(id)
    reqPrenom = "SELECT prenomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_prenom = pd.read_sql(reqPrenom, conn)
    W_prenom = W_prenom.iloc[0]["prenomJoueur"]
    reqNom = "SELECT nomJoueur FROM JOUEURS WHERE idNHL = " + strId
    W_nom = pd.read_sql(reqNom, conn)
    W_nom = W_nom.iloc[0]["nomJoueur"]
    return W_prenom + " " + W_nom

#ajouter l'échange au registre
def ajouterRegistre(dat, condition):
    #générer le idTrade
    req = "SELECT MAX(idTrade) FROM ÉCHANGES"
    m = pd.read_sql(req, conn)
    W_idTrade = m.iloc[0]["MAX(idTrade)"] + 1
    #générer EquipeA
    equipes = tradeJoueurs["equipeRecoit"].unique()
    equipeA = equipes[0]
    #générer JoueursA
    is_equA = tradeJoueurs["equipeRecoit"] == equipeA
    joueursA = tradeJoueurs[is_equA]
    joueursA = joueursA["idNHL"]
    a = []
    for i in joueursA:
        nom = getNom(i)
        a.__iadd__([nom])
    str_a = ', '.join([str(elem) for elem in a])
    #générer EquipeB
    equipeB = equipes[1]
    #générer JoueursB
    is_equB = tradeJoueurs["equipeRecoit"] == equipeB
    joueursB = tradeJoueurs[is_equB]
    joueursB = joueursB["idNHL"]
    b = []
    for i in joueursB:
        nom = getNom(i)
        b.__iadd__([nom])
    str_b = ', '.join([str(elem) for elem in b])
    #générer ChoixA


    #générer ChoixB



    d = {"idTrade" : [W_idTrade], "DateTrade" : [dat], "EquipeA" : [equipeA], "JoueursA" : [str_a], "EquipeB" : [equipeB], "JoueursB" : [str_b], "Condition" : [condition]}
    echange = pd.DataFrame(data=d)
    echange.to_sql("ÉCHANGES", conn, if_exists="append", index=False)
    ok = "ok"
    return echange

def echange(joueurs, dat, condition="vide"):
    for i in joueurs:
        nouvellePeriode(i, dat)
    finirPeriode(joueurs, dat)
    #fonction pour faire le switch des choix
    done = "done"
    ajouterRegistre(dat, condition)
    return done

#pour les choix:
    #fonction pour les choix va être inclue dans echange() et dans ajouterRegistre.

In [149]:
dat = "2021-02-19"

#echange(ids, dat)

'done'